In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold, StratifiedGroupKFold, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score, recall_score, f1_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
import joblib

os.chdir('..')
print(os.getcwd())

### CSV loading

In [ ]:
# # df = pd.read_csv('opensmile.csv')
# df = pd.read_csv('opensmile_standardized.csv')
# # df = shuffle(df)
# # df.reset_index(inplace=True, drop=True)
# # df.drop('voiceID', inplace = True, axis = 1)
# df['label'].value_counts()
# df['gender'].value_counts()
# df['age'].value_counts()
# df

In [ ]:
# x = df.iloc[:, 4:].values  # 特征
# y = df['label'].values  # 标签
# speakers = df['speaker'].values  # 说话人信息

# kf = StratifiedGroupKFold(n_splits=5, shuffle=True)

# X_train_folds = []
# Y_train_folds = []
# X_test_folds = []
# Y_test_folds = []
# speaker_train_folds = []  # 用来保存每个训练折对应的说话人
# speaker_test_folds = []  # 用来保存每个测试折对应的说话人

# for train_index, test_index in kf.split(x, y, speakers):
#     X_train_folds.append(x[train_index])
#     Y_train_folds.append(y[train_index])
#     X_test_folds.append(x[test_index])
#     Y_test_folds.append(y[test_index])
    
#     # 保存每个折中的说话人信息
#     speaker_train_folds.append(speakers[train_index])
#     speaker_test_folds.append(speakers[test_index])

# # 保存数据
# for i in range(5):
#     # 保存训练集和测试集数据
#     np.save(f'./X_train_{i}.npy', X_train_folds[i])
#     np.save(f'./Y_train_{i}.npy', Y_train_folds[i])
#     np.save(f'./X_test_{i}.npy', X_test_folds[i])
#     np.save(f'./Y_test_{i}.npy', Y_test_folds[i])
    
#     # 保存每个折的说话人信息
#     # np.save(f'./speaker_train_{i}.npy', speaker_train_folds[i])
#     # np.save(f'./speaker_test_{i}.npy', speaker_test_folds[i])
#     # 保存每个折的说话人信息到txt文件
#     with open(f'./speaker_train_{i}.txt', 'w') as f_train:
#         for speaker in speaker_train_folds[i]:
#             f_train.write(f'{speaker}\n')  # 每个说话人信息写一行
    
#     with open(f'./speaker_test_{i}.txt', 'w') as f_test:
#         for speaker in speaker_test_folds[i]:
#             f_test.write(f'{speaker}\n')  # 每个说话人信息写一行



In [ ]:
X_train = np.load('./X_all_train_mutual.npy')
Y_train = np.load('./Y_train.npy')
X_test = np.load('./X_all_test_mutual.npy')
Y_test = np.load('./Y_test.npy')

X_train, Y_train = shuffle(X_train, Y_train, random_state=42)
X_test, Y_test = shuffle(X_test, Y_test, random_state=42)


### Feature scaling

In [ ]:
# # Scale
# sc = StandardScaler()

# # sc = MinMaxScaler()
# # sc = joblib.load(scaler.pkl")

# # sc = RobustScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

# # 保存数据尺度参数
# joblib.dump(sc, "scaler.pkl")

# # pd.DataFrame(X_test)

# KNN

### Hyperparameter tuning for KNN

In [ ]:
# 设置超参数
leaf_size = list(range(1, 50))
n_neighbors = list(range(1, 20))
p = [1, 2]
knn_hyperparameters = dict(leaf_size=leaf_size, n_neighbors=n_neighbors, p=p)

# 定义KNN模型
knn = KNeighborsClassifier()

# 使用已有的交叉验证（不再在GridSearchCV中重复设置cv）
knn_clf = GridSearchCV(knn, knn_hyperparameters)  # 移除cv=10

# 拟合模型
knn_best_model = knn_clf.fit(X_train, Y_train)

# 保存最佳模型
joblib.dump(knn_best_model, "./knn.pkl")

# 输出最佳超参数和评分
print('Best leaf_size:       ', knn_best_model.best_estimator_.get_params()['leaf_size'])
print('Best p:               ', knn_best_model.best_estimator_.get_params()['p'])
print('Best n_neighbors:     ', knn_best_model.best_estimator_.get_params()['n_neighbors'])
print('Best Score:           %s' % knn_best_model.best_score_)
print('Best Hyperparameters: %s' % knn_best_model.best_params_)

# 预测测试集
y_pred_knn = knn_best_model.predict(X_test)

# 计算混淆矩阵和性能指标
knn_conf_matrix = confusion_matrix(Y_test, y_pred_knn)
tn, fp, fn, tp = knn_conf_matrix.ravel()
knn_sen = tp / (tp + fn)
knn_spe = tn / (tn + fp)

# 输出结果
print(f"ConfusionMatrix: \n {knn_conf_matrix}")      
print('Accuracy: ', accuracy_score(Y_test, y_pred_knn))  # 准确率
print(f'Specificity: {knn_spe}')  # 特异度
print(f'Sensitivity:  {knn_sen}')  # 敏感度
print('F1-score: ', f1_score(Y_test, y_pred_knn, average='binary'))  # F1分数
print('Recall: ', recall_score(Y_test, y_pred_knn, average='binary'))  # 召回率
print('Precision: ', precision_score(Y_test, y_pred_knn, average='binary'))  # 精确度

# Decision Trees

### Hyperparameter tuning for Decision Trees

In [ ]:
# 设置超参数
max_depth = list(range(1, 10))
min_samples_split = list(range(2, 10))
min_samples_leaf = list(range(1, 5))
criterion = ['gini', 'entropy']
dt_hyperparameters = dict(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf, criterion=criterion)

# 定义决策树模型
dt = tree.DecisionTreeClassifier()

# 使用GridSearchCV来进行超参数选择，不需要指定cv参数
dt_clf = GridSearchCV(dt, dt_hyperparameters, refit=True)  # 移除cv=10

# 拟合模型
dt_best_model = dt_clf.fit(X_train, Y_train)

# 保存最佳模型
joblib.dump(dt_best_model, "dt.pkl")

# 输出最佳超参数和评分
print('Best Score:           %s' % dt_best_model.best_score_)
print('Best Hyperparameters: %s' % dt_best_model.best_params_)

# 预测测试集
y_pred_dt = dt_best_model.predict(X_test)

# 计算混淆矩阵和性能指标
dt_conf_matrix = confusion_matrix(Y_test, y_pred_dt)
tn, fp, fn, tp = dt_conf_matrix.ravel()
dt_sen = tp / (tp + fn)
dt_spe = tn / (tn + fp)

# 输出结果
print(f"ConfusionMatrix: \n {dt_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_dt))  # 准确率
print(f'Specificity: {dt_spe}')  # 特异度
print(f'Sensitivity:  {dt_sen}')  # 敏感度
print('F1-score: ', f1_score(Y_test, y_pred_dt, average='binary'))  # F1分数
print('Recall: ', recall_score(Y_test, y_pred_dt, average='binary'))  # 召回率
print('Precision: ', precision_score(Y_test, y_pred_dt, average='binary'))  # 精确度

# SVM

### Hyperparameter tuning for SVM

In [ ]:
# 设置超参数
C = [0.1, 1, 10, 100, 1000]
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
# rbf -> 高斯核，poly -> 多项式核，linear -> 线性核，sigmoid -> sigmoid核
kernel = ['rbf', 'poly', 'linear', 'sigmoid']
svm_hyperparameters = dict(C=C, gamma=gamma, kernel=kernel)

# 定义SVM模型
svm2 = svm.SVC(probability=True)

# 使用GridSearchCV来进行超参数选择，不需要指定cv参数
svm_clf = GridSearchCV(svm2, svm_hyperparameters, refit=True)  # 移除cv=10

# 拟合模型
svm_best_model = svm_clf.fit(X_train, Y_train)

# 保存最佳模型
joblib.dump(svm_best_model, "svm.pkl")

# 输出最佳超参数和评分
print('Best Score:           %s' % svm_best_model.best_score_)
print('Best Hyperparameters: %s' % svm_best_model.best_params_)

# 预测测试集
y_pred_svm = svm_best_model.predict(X_test)

# 计算混淆矩阵和性能指标
svm_conf_matrix = confusion_matrix(Y_test, y_pred_svm)
tn, fp, fn, tp = svm_conf_matrix.ravel()
svm_sen = tp / (tp + fn)
svm_spe = tn / (tn + fp)

# 输出结果
print(f"ConfusionMatrix: \n {svm_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_svm))  # 准确率
print(f'Specificity: {svm_spe}')  # 特异度
print(f'Sensitivity:  {svm_sen}')  # 敏感度
print('F1-score: ', f1_score(Y_test, y_pred_svm, average='binary'))  # F1分数
print('Recall: ', recall_score(Y_test, y_pred_svm, average='binary'))  # 召回率
print('Precision: ', precision_score(Y_test, y_pred_svm, average='binary'))  # 精确度


# AdaBoost

### Hyperparameters tuning for AdaBoost

In [ ]:
# 设置超参数
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
learning_rate = [1, 0.5, 0.25, 0.1, 0.05, 0.01]
abc_hyperparameters = dict(n_estimators=n_estimators, learning_rate=learning_rate)

# 定义AdaBoost模型
abc_model = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=1))

# 使用GridSearchCV进行超参数选择，但不再需要cv参数
abc_clf = GridSearchCV(abc_model, abc_hyperparameters, refit=True)  # 移除cv参数

# 拟合模型
abc_best_model = abc_clf.fit(X_train, Y_train)

# 保存最佳模型
joblib.dump(abc_best_model, "ab.pkl")

# 输出最佳超参数和评分
print('Best Score:           %s' % abc_best_model.best_score_)
print('Best Hyperparameters: %s' % abc_best_model.best_params_)

# 预测测试集
y_pred_abc = abc_best_model.predict(X_test)

# 计算混淆矩阵和性能指标
abc_conf_matrix = confusion_matrix(Y_test, y_pred_abc)
tn, fp, fn, tp = abc_conf_matrix.ravel()
abc_sen = tp / (tp + fn)
abc_spe = tn / (tn + fp)

# 输出结果
print(f"ConfusionMatrix: \n {abc_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_abc))  # 准确率
print(f'Specificity: {abc_spe}')  # 特异度
print(f'Sensitivity:  {abc_sen}')  # 敏感度
print('Recall: ', recall_score(Y_test, y_pred_abc, average='binary'))  # 召回率
print('F1-score: ', f1_score(Y_test, y_pred_abc, average='binary'))  # F1分数
print('Precision: ', precision_score(Y_test, y_pred_abc, average='binary'))  # 精确度


# Random Forest

### Hyperparameters tuning for RF

In [ ]:
# 设置超参数
n_estimators = [1, 2, 4, 8, 16, 32, 64, 100, 200]
max_depth = [1, 3, 5, 7, 9]
max_features = ['auto', 'sqrt', 'log2']
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 5, 10, 15]

rf_hyperparameters = dict(n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, 
                          min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

# 创建随机森林分类器
rf_model = RandomForestClassifier()

# 使用GridSearchCV进行超参数选择，移除cv参数
rf_clf = GridSearchCV(rf_model, rf_hyperparameters, refit=True)  # 移除cv参数

# 拟合模型
rf_best_model = rf_clf.fit(X_train, Y_train)

# 保存最佳模型
joblib.dump(rf_best_model, "rf.pkl")

# 输出最佳超参数和评分
print('Best Score:           %s' % rf_best_model.best_score_)
print('Best Hyperparameters: %s' % rf_best_model.best_params_)

# 预测测试集
y_pred_rf = rf_best_model.predict(X_test)

# 计算混淆矩阵和性能指标
rf_conf_matrix = confusion_matrix(Y_test, y_pred_rf)
tn, fp, fn, tp = rf_conf_matrix.ravel()
rf_sen = tp / (tp + fn)
rf_spe = tn / (tn + fp)

# 输出结果
print(f"ConfusionMatrix: \n {rf_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_rf))  # 准确率
print(f'Specificity: {rf_spe}')  # 特异度
print(f'Sensitivity:  {rf_sen}')  # 敏感度
print('Recall: ', recall_score(Y_test, y_pred_rf, average='binary'))  # 召回率
print('F1-score: ', f1_score(Y_test, y_pred_rf, average='binary'))  # F1分数
print('Precision: ', precision_score(Y_test, y_pred_rf, average='binary'))  # 精确度


In [ ]:
import xgboost as xgb

# 设置超参数
n_estimators = [50, 100, 200, 300]
learning_rate = [0.1, 0.01, 0.05, 0.3]
max_depth = [3, 6, 9, 12]
subsample = [0.7, 0.8, 0.9, 1.0]
colsample_bytree = [0.7, 0.8, 1.0]

xgb_hyperparameters = dict(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth,
                           subsample=subsample, colsample_bytree=colsample_bytree)

# 创建XGBoost分类器
xgb_model = xgb.XGBClassifier()

# 使用GridSearchCV进行超参数选择，移除cv参数
xgb_clf = GridSearchCV(xgb_model, xgb_hyperparameters, refit=True)

# 拟合模型
xgb_best_model = xgb_clf.fit(X_train, Y_train)

# 保存最佳模型
joblib.dump(xgb_best_model, "xgb.pkl")

# 输出最佳超参数和评分
print('Best Score:           %s' % xgb_best_model.best_score_)
print('Best Hyperparameters: %s' % xgb_best_model.best_params_)

# 预测测试集
y_pred_xgb = xgb_best_model.predict(X_test)

# 计算混淆矩阵和性能指标
xgb_conf_matrix = confusion_matrix(Y_test, y_pred_xgb)
tn, fp, fn, tp = xgb_conf_matrix.ravel()
xgb_sen = tp / (tp + fn)
xgb_spe = tn / (tn + fp)

# 输出结果
print(f"ConfusionMatrix: \n {xgb_conf_matrix}")      
print('Accuracy:  ', accuracy_score(Y_test, y_pred_xgb))  # 准确率
print(f'Specificity: {xgb_spe}')  # 特异度
print(f'Sensitivity:  {xgb_sen}')  # 敏感度
print('Recall: ', recall_score(Y_test, y_pred_xgb, average='binary'))  # 召回率
print('F1-score: ', f1_score(Y_test, y_pred_xgb, average='binary'))  # F1分数
print('Precision: ', precision_score(Y_test, y_pred_xgb, average='binary'))  # 精确度
